In [ ]:
!wget http://www.soshnikov.com/permanent/data/petfaces.tar.gz
!tar xfz petfaces.tar.gz
!rm petfaces.tar.gz

--2024-04-25 09:37:05--  http://www.soshnikov.com/permanent/data/petfaces.tar.gz
Resolving www.soshnikov.com (www.soshnikov.com)... 79.137.227.122
Connecting to www.soshnikov.com (www.soshnikov.com)|79.137.227.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24483412 (23M) [application/x-tar]
Saving to: ‘petfaces.tar.gz’

petfaces.tar.gz     100%[===================>]  23.35M   121KB/s    in 3m 24s  

2024-10-08 18:16:03 (117 KB/s) - ‘petfaces.tar.gz’ saved [24483412/24483412]



In [ ]:
pip install split-folders

In [ ]:
pip install torchinfo

In [ ]:
pip install torchmetrics

In [ ]:
pip install wget

In [ ]:
import numpy as np
import pandas as pd
import os
import wget
import matplotlib.pyplot as plt
from PIL import Image
from mlxtend.plotting import plot_confusion_matrix

from timeit import default_timer as timer
from tqdm.notebook import tqdm
from typing import Tuple, List, Dict

import random
import pathlib
import splitfolders

import torch
import torchvision
import torch.nn as nn

from torchinfo import summary

from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid


from torchmetrics import ConfusionMatrix
from sklearn.metrics import classification_report

In [ ]:
SEED = 100
BATCH_SIZE = 64
N_WORKERS = os.cpu_count()

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
if os.path.isdir('data/') == False:
    os.mkdir('data/')

In [ ]:
path_data ='data/petfaces'

In [ ]:
if os.path.isdir(path_data) == False:
    os.chdir('data/')

    wget.download('http://www.soshnikov.com/permanent/data/petfaces.tar.gz')

    !tar xfz petfaces.tar.gz
    os.remove('petfaces.tar.gz')

    os.chdir('..')

In [ ]:
SIZE = 64
BATCH_SIZE = 16
N_WORKERS = os.cpu_count()
SAMPLE_SIZE = 100

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [ ]:
def plot_transformed_images(transforms: torchvision.transforms.transforms.Compose,
                            n_images: int,
                            target_dir: str,
                            seed: int = None,
                            depth: str = '*/*/*') -> None:

    font_s = 12
    random.seed(seed)

    image_paths = list(pathlib.Path(target_dir).glob(f'{depth}.jpg'))
    image_random = random.sample(image_paths, k=n_images)

    for image in image_random:
        with Image.open(image) as file:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(file)
            ax[0].set_title(f'Исходный: \n{np.array(file).shape}')
            ax[0].axis('off')
            image_transformed = transforms(file).permute(1, 2, 0)
            ax[1].imshow(image_transformed)
            ax[1].set_title(f'Трансформированный: \n{image_transformed.numpy().shape}')
            ax[1].axis('off')

        fig.suptitle(f'Класс {image.parent.stem}', fontsize=font_s+4)

In [ ]:
train_data = datasets.ImageFolder(
    root=path_data,
    transform=data_transforms
)

In [ ]:
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
    shuffle=True,
    drop_last=True
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, sample_size: int) -> None:
        super(Generator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(sample_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )

        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)
        )

        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )

        self.conv5 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        return x

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__

    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)

    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [ ]:
generator = Generator(SAMPLE_SIZE).apply(weights_init).to(device)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        return x

In [ ]:
discriminator = Discriminator().apply(weights_init).to(device)

In [ ]:
class GAN:

    def __init__(self, generator, discriminator, sample_size: int, device: str = device) -> None:
        self.generator = generator
        self.discriminator = discriminator
        self.sample_size = sample_size
        self.device = device

        self.fixed_noise = torch.randn(16, self.sample_size, 1, 1, device=self.device)

        self.real_label = 1.
        self.fake_label = 0.


    def train(self,
              n_epochs: int,
              dataloader: torch.utils.data.dataloader.DataLoader,
              criterion: torch.nn.modules.loss,
              optim_gen: torch.optim,
              optim_dis: torch.optim,
              img_title: str) -> None:

        self.img_title = img_title
        self.__criterion = criterion
        self.__optim_gen = optim_gen
        self.__optim_dis = optim_dis

        self.images = []
        self.loss_generator = []
        self.loss_discriminator = []


        for epoch in tqdm(range(n_epochs), 'Training process'):
            for i, data in enumerate(dataloader, 0):

                pred_dis, pred_gen_before, loss_dis = self.train_discriminator(data)
                loss_gen, pred_gen_after = self.train_generator()

                self.loss_generator.append(loss_gen)
                self.loss_discriminator.append(loss_dis)

                if i == 0 or i % 200 == 0 or i == len(dataloader) - 1:
                    self.print_iter(epoch, n_epochs, i, len(dataloader) - 1,
                                    loss_dis, loss_gen, pred_dis, pred_gen_before, pred_gen_after)

                self.generator.eval()
                with torch.no_grad():
                    fake_images = self.generator(self.fixed_noise).cpu()






    def print_iter(self, *args):
        print('[%d/%d][%d/%d]\tLoss Dis: %.4f\tLoss Gen: %.4f\tPred Dis: %.4f\tPred Gen: %.4f / %.4f'
              % (args))


    def train_discriminator(self, data: list) -> Tuple[float, float, float]:
        self.discriminator.zero_grad()

        real_images = data[0].to(self.device)
        b_size = real_images.size(0)
        self.__label = torch.full((b_size,), self.real_label, dtype=torch.float, device=self.device)


        pred_real = self.discriminator(real_images).view(-1)
        loss_real = self.__criterion(pred_real, self.__label)


        loss_real.backward()
        pred_dis = pred_real.mean().item()

        noise = torch.randn(b_size, self.sample_size, 1, 1, device=self.device)
        self.__fake_images = self.generator(noise)
        self.__label.fill_(self.fake_label)


        pred_fake = self.discriminator(self.__fake_images.detach()).view(-1)
        loss_fake = self.__criterion(pred_fake, self.__label)


        loss_fake.backward()
        pred_gen_before = pred_fake.mean().item()

        loss = loss_real + loss_fake
        self.__optim_dis.step()

        return pred_dis, pred_gen_before, loss.item()


    def train_generator(self) -> Tuple[float, float]:
        self.generator.zero_grad()

        self.__label.fill_(self.real_label)


        pred = self.discriminator(self.__fake_images).view(-1)
        loss = self.__criterion(pred, self.__label)

        loss.backward()
        pred_gen_after = pred.mean().item()

        self.__optim_gen.step()

        return loss.item(), pred_gen_after


In [ ]:
N_EPOCHS = 100
CRITERION = nn.BCELoss()

LEARN_R = 2e-4
BETAS = (0.5, 0.999)

OPTIM_DIS = torch.optim.Adam(discriminator.parameters(), lr=LEARN_R, betas=BETAS)
OPTIM_GEN = torch.optim.Adam(generator.parameters(), lr=LEARN_R, betas=BETAS)

In [ ]:
gan_64 = GAN(generator, discriminator, SAMPLE_SIZE)

In [ ]:
gan_64.train(
    n_epochs=N_EPOCHS,
    dataloader=train_dataloader,
    criterion=CRITERION,
    optim_gen=OPTIM_GEN,
    optim_dis=OPTIM_DIS,
    img_title='gan_64'
)

Training process:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[0/100][0/199]	Loss Dis: 1.8754	Loss Gen: 8.0383	Pred Dis: 0.6865	Pred Gen: 0.6843 / 0.0005


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[0/100][199/199]	Loss Dis: 0.0018	Loss Gen: 39.9960	Pred Dis: 0.9982	Pred Gen: 0.0000 / 0.0000
[1/100][0/199]	Loss Dis: 0.0000	Loss Gen: 39.5310	Pred Dis: 1.0000	Pred Gen: 0.0000 / 0.0000
[1/100][199/199]	Loss Dis: 0.0000	Loss Gen: 38.6203	Pred Dis: 1.0000	Pred Gen: 0.0000 / 0.0000
[2/100][0/199]	Loss Dis: 0.0000	Loss Gen: 39.1109	Pred Dis: 1.0000	Pred Gen: 0.0000 / 0.0000
[2/100][199/199]	Loss Dis: 0.3300	Loss Gen: 3.0970	Pred Dis: 0.7941	Pred Gen: 0.0605 / 0.0639
[3/100][0/199]	Loss Dis: 0.2549	Loss Gen: 2.7777	Pred Dis: 0.8718	Pred Gen: 0.0924 / 0.0872
[3/100][199/199]	Loss Dis: 0.4527	Loss Gen: 4.5145	Pred Dis: 0.7097	Pred Gen: 0.0347 / 0.0275
[4/100][0/199]	Loss Dis: 0.3650	Loss Gen: 3.0340	Pred Dis: 0.7807	Pred Gen: 0.0632 / 0.0674
[4/100][199/199]	Loss Dis: 1.2069	Loss Gen: 2.7543	Pred Dis: 0.5177	Pred Gen: 0.1425 / 0.0939
[5/100][0/199]	Loss Dis: 0.1564	Loss Gen: 3.9070	Pred Dis: 0.9699	Pred Gen: 0.1122 / 0.0349
[5/100][199/199]	Loss Dis: 0.4915	Loss Gen: 3.0232	Pred Dis: 0.788

#GAN 128x128

In [ ]:
SIZE = 128
BATCH_SIZE = 16
N_WORKERS = os.cpu_count()
SAMPLE_SIZE = 100

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [ ]:
train_data = datasets.ImageFolder(
    root=path_data,
    transform=data_transforms
)

In [ ]:
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
    shuffle=True,
    drop_last=True
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, sample_size: int) -> None:
        super(Generator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(sample_size, 1024, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True)
        )

        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )

        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)
        )

        self.conv5 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )

        self.conv6 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)

        return x

In [ ]:
generator = Generator(SAMPLE_SIZE).apply(weights_init).to(device)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)

        return x

In [ ]:
discriminator = Discriminator().apply(weights_init).to(device)

In [ ]:
class GAN:

    def __init__(self, generator, discriminator, sample_size: int, device: str = device) -> None:
        self.generator = generator
        self.discriminator = discriminator
        self.sample_size = sample_size
        self.device = device

        self.fixed_noise = torch.randn(16, self.sample_size, 1, 1, device=self.device)

        self.real_label = 1.
        self.fake_label = 0.


    def train(self,
              n_epochs: int,
              dataloader: torch.utils.data.dataloader.DataLoader,
              criterion: torch.nn.modules.loss,
              optim_gen: torch.optim,
              optim_dis: torch.optim,
              img_title: str) -> None:

        self.img_title = img_title
        self.__criterion = criterion
        self.__optim_gen = optim_gen
        self.__optim_dis = optim_dis

        self.images = []
        self.loss_generator = []
        self.loss_discriminator = []

        for epoch in tqdm(range(n_epochs), 'Training process'):
            for i, data in enumerate(dataloader, 0):

                pred_dis, pred_gen_before, loss_dis = self.train_discriminator(data)
                loss_gen, pred_gen_after = self.train_generator()

                self.loss_generator.append(loss_gen)
                self.loss_discriminator.append(loss_dis)

                if i == 0 or i % 200 == 0 or i == len(dataloader) - 1:
                    self.print_iter(epoch, n_epochs, i, len(dataloader) - 1,
                                    loss_dis, loss_gen, pred_dis, pred_gen_before, pred_gen_after)

                self.generator.eval()
                with torch.no_grad():
                    fake_images = self.generator(self.fixed_noise).cpu()

                self.images.append(make_grid(fake_images, 4, padding=2, normalize=True))



    def print_iter(self, *args):
        print('[%d/%d][%d/%d]\tLoss Dis: %.4f\tLoss Gen: %.4f\tPred Dis: %.4f\tPred Gen: %.4f / %.4f'
              % (args))


    def train_discriminator(self, data: list) -> Tuple[float, float, float]:
        self.discriminator.zero_grad()

        real_images = data[0].to(self.device)
        b_size = real_images.size(0)
        self.__label = torch.full((b_size,), self.real_label, dtype=torch.float, device=self.device)

        # forward
        pred_real = self.discriminator(real_images).view(-1)
        loss_real = self.__criterion(pred_real, self.__label)

        # backward
        loss_real.backward()
        pred_dis = pred_real.mean().item()

        noise = torch.randn(b_size, self.sample_size, 1, 1, device=self.device)
        self.__fake_images = self.generator(noise)
        self.__label.fill_(self.fake_label)

        # forward
        pred_fake = self.discriminator(self.__fake_images.detach()).view(-1)
        loss_fake = self.__criterion(pred_fake, self.__label)

        # backward
        loss_fake.backward()
        pred_gen_before = pred_fake.mean().item()

        loss = loss_real + loss_fake
        self.__optim_dis.step()

        return pred_dis, pred_gen_before, loss.item()

    def train_generator(self) -> Tuple[float, float]:
        self.generator.zero_grad()

        self.__label.fill_(self.real_label)

        # forward
        pred = self.discriminator(self.__fake_images).view(-1)
        loss = self.__criterion(pred, self.__label)

        # backward
        loss.backward()
        pred_gen_after = pred.mean().item()

        self.__optim_gen.step()

        return loss.item(), pred_gen_after

In [ ]:
gan_128 = GAN(generator, discriminator, SAMPLE_SIZE)

In [ ]:
gan_128.train(
    n_epochs=N_EPOCHS,
    dataloader=train_dataloader,
    criterion=CRITERION,
    optim_gen=OPTIM_GEN,
    optim_dis=OPTIM_DIS,
    img_title='gan_128'
)

Training process:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[0/100][0/199]	Loss Dis: 1.1601	Loss Gen: 0.8927	Pred Dis: 0.6686	Pred Gen: 0.4649 / 0.4649
[0/100][199/199]	Loss Dis: 1.3528	Loss Gen: 0.9189	Pred Dis: 0.5647	Pred Gen: 0.4644 / 0.4644
[1/100][0/199]	Loss Dis: 1.4278	Loss Gen: 0.8532	Pred Dis: 0.5656	Pred Gen: 0.4940 / 0.4940
[1/100][199/199]	Loss Dis: 0.9559	Loss Gen: 1.1962	Pred Dis: 0.6500	Pred Gen: 0.3451 / 0.3451
[2/100][0/199]	Loss Dis: 1.2529	Loss Gen: 0.7538	Pred Dis: 0.6478	Pred Gen: 0.5026 / 0.5026
[2/100][199/199]	Loss Dis: 1.0613	Loss Gen: 1.0104	Pred Dis: 0.6513	Pred Gen: 0.4152 / 0.4152
[3/100][0/199]	Loss Dis: 1.4033	Loss Gen: 0.9072	Pred Dis: 0.5918	Pred Gen: 0.4625 / 0.4625
[3/100][199/199]	Loss Dis: 1.0993	Loss Gen: 0.9475	Pred Dis: 0.7028	Pred Gen: 0.4457 / 0.4457
[4/100][0/199]	Loss Dis: 1.3269	Loss Gen: 0.8166	Pred Dis: 0.6358	Pred Gen: 0.4944 / 0.4944
[4/100][199/199]	Loss Dis: 1.3045	Loss Gen: 0.7985	Pred Dis: 0.6253	Pred Gen: 0.4958 / 0.4958
[5/100][0/199]	Loss Dis: 1.1494	Loss Gen: 0.9444	Pred Dis: 0.6340	Pred

#GAN 264x264

In [ ]:
SIZE = 256
BATCH_SIZE = 16
N_WORKERS = os.cpu_count()
SAMPLE_SIZE = 100

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [ ]:
train_data = datasets.ImageFolder(
    root=path_data,
    transform=data_transforms
)

In [ ]:
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
    shuffle=True,
    drop_last=True
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, sample_size: int) -> None:
        super(Generator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(sample_size, 2048, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(2048),
            nn.ReLU(True)
        )

        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(2048, 1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True)
        )

        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )

        self.conv5 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)
        )

        self.conv5 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )

        self.conv7 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)

        return x

In [ ]:
generator = Generator(SAMPLE_SIZE).apply(weights_init).to(device)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(1024, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = self.conv1(z)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)

        return x

In [ ]:
discriminator = Discriminator().apply(weights_init).to(device)

In [ ]:
class GAN:

    def __init__(self, generator, discriminator, sample_size: int, device: str = device) -> None:
        self.generator = generator
        self.discriminator = discriminator
        self.sample_size = sample_size
        self.device = device

        self.fixed_noise = torch.randn(32, self.sample_size, 1, 1, device=self.device)

        self.real_label = 1.
        self.fake_label = 0.


    def train(self,
              n_epochs: int,
              dataloader: torch.utils.data.dataloader.DataLoader,
              criterion: torch.nn.modules.loss,
              optim_gen: torch.optim,
              optim_dis: torch.optim,
              img_title: str) -> None:

        self.img_title = img_title
        self.__criterion = criterion
        self.__optim_gen = optim_gen
        self.__optim_dis = optim_dis

        self.images = []
        self.loss_generator = []
        self.loss_discriminator = []

        for epoch in tqdm(range(n_epochs), 'Training process'):
            for i, data in enumerate(dataloader, 0):

                pred_dis, pred_gen_before, loss_dis = self.train_discriminator(data)
                loss_gen, pred_gen_after = self.train_generator()

                self.loss_generator.append(loss_gen)
                self.loss_discriminator.append(loss_dis)

                if i == 0 or i % 200 == 0 or i == len(dataloader) - 1:
                    self.print_iter(epoch, n_epochs, i, len(dataloader) - 1,
                                    loss_dis, loss_gen, pred_dis, pred_gen_before, pred_gen_after)

                self.generator.eval()
                with torch.no_grad():
                    fake_images = self.generator(self.fixed_noise).cpu()

                self.images.append(make_grid(fake_images, 4, padding=2, normalize=True))



    def print_iter(self, *args):
        print('[%d/%d][%d/%d]\tLoss Dis: %.4f\tLoss Gen: %.4f\tPred Dis: %.4f\tPred Gen: %.4f / %.4f'
              % (args))


    def train_discriminator(self, data: list) -> Tuple[float, float, float]:
        self.discriminator.zero_grad()

        real_images = data[0].to(self.device)
        b_size = real_images.size(0)
        self.__label = torch.full((b_size,), self.real_label, dtype=torch.float, device=self.device)

        # forward
        pred_real = self.discriminator(real_images).view(-1)
        loss_real = self.__criterion(pred_real, self.__label)

        # backward
        loss_real.backward()
        pred_dis = pred_real.mean().item()

        noise = torch.randn(b_size, self.sample_size, 1, 1, device=self.device)
        self.__fake_images = self.generator(noise)
        self.__label.fill_(self.fake_label)

        # forward
        pred_fake = self.discriminator(self.__fake_images.detach()).view(-1)
        loss_fake = self.__criterion(pred_fake, self.__label)

        # backward
        loss_fake.backward()
        pred_gen_before = pred_fake.mean().item()

        loss = loss_real + loss_fake
        self.__optim_dis.step()

        return pred_dis, pred_gen_before, loss.item()

    def train_generator(self) -> Tuple[float, float]:
        self.generator.zero_grad()

        self.__label.fill_(self.real_label)

        # forward
        pred = self.discriminator(self.__fake_images).view(-1)
        loss = self.__criterion(pred, self.__label)

        # backward
        loss.backward()
        pred_gen_after = pred.mean().item()

        self.__optim_gen.step()

        return loss.item(), pred_gen_after

In [ ]:
N_EPOCHS = 100
CRITERION = nn.BCELoss()

LEARN_R = 2e-4
BETAS = (0.5, 0.999)

OPTIM_DIS = torch.optim.Adam(discriminator.parameters(), lr=LEARN_R, betas=BETAS)
OPTIM_GEN = torch.optim.Adam(generator.parameters(), lr=LEARN_R, betas=BETAS)

In [ ]:
gan_256 = GAN(generator, discriminator, SAMPLE_SIZE)

In [ ]:
gan_256.train(
    n_epochs=N_EPOCHS,
    dataloader=train_dataloader,
    criterion=CRITERION,
    optim_gen=OPTIM_GEN,
    optim_dis=OPTIM_DIS,
    img_title='gan_64'
)

Training process:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


RuntimeError: Given transposed=1, weight of size [128, 64, 4, 4], expected input[16, 256, 32, 32] to have 128 channels, but got 256 channels instead